In [ ]:
%run helper.py
name = 'hemisphere_5t'
input_path = '../../data/{}.json.gz'.format(name)

io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path, handleBoundary=False, handlePivots = True)
rod_colors = get_color_field(curr_um, input_data)

def get_saddle_view(umbrella):
    import mesh
    view = linkage_vis.LinkageViewerWithSurface(umbrella, target_mesh, width=1024, height=600)
    set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
    view.averagedMaterialFrames = True
    view.setCameraParams(((-11.144512556513881, -11.281582930201559, 4.961758097391448),
                             (0.2578692282155091, 0.4699669630554463, 0.8441768267229215),
                             (4.75386274113031, -3.697285366936124, -4.116992725883979)))
    view.showScalarField(rod_colors)
    return view

In [ ]:
curr_um = pickle.load(gzip.open("../../output/hemisphere_5t_optimized_rest_state_equilibrium_2022_04_25_11_08_target_height_factor_5.0.pkl.gz", 'r'))

In [ ]:
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
view = get_saddle_view(curr_um)
view.show()

In [ ]:
break_input_angle_symmetry(curr_um)

In [ ]:
def eqm_callback(prob, i):
    if (i % 1 == 0):
        view.showScalarField(rod_colors)

In [ ]:
configure_umbrella_pre_deployment(curr_um, thickness, target_height_multiplier)

In [ ]:
results = staged_deployment(curr_um, np.logspace(-4, 0, 5), eqm_callback, OPTS, fixedVars, elasticEnergyIncreaseFactorLimit = 1.5)

## Stiffness analysis

$$\frac{\partial^2{U}}{\partial{\vec{x}}^2}\vec{v} = \lambda M \vec{v}$$


#### Update target plate separation

In [ ]:
configure_umbrella_pre_deployment(curr_um, thickness, 10)
# curr_um.attractionWeight = 1e-3
curr_um.attractionWeight = 0
OPTS.niter = 100

with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

results.success

#### Compute vibrational modes

In [ ]:
import compute_vibrational_modes

In [ ]:
class ModalAnalysisWrapper:
    def __init__(self, um):
        self.um = um
    def hessian(self):
        return self.um.hessian(umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Full)
    def massMatrix(self): return self.um.massMatrix()
    def lumpedMassMatrix(self): return self.um.lumpedMassMatrix()

In [ ]:
lambdas, modes = compute_vibrational_modes.compute_vibrational_modes(ModalAnalysisWrapper(curr_um), fixedVars=curr_um.rigidJointAngleDoFIndices(), mtype=compute_vibrational_modes.MassMatrixType.FULL, n=16, sigma=-1e-6)

import mode_viewer, importlib
importlib.reload(mode_viewer);
mview = mode_viewer.ModeViewer(curr_um, modes, lambdas, amplitude=0.5 / lambdas[6])
# mview.showScalarField(rod_colors)
mview.show()

In [ ]:
lambdas[6]

In [ ]:
lambdas[6]

In [ ]:
lambdas[6]

## Summary

In [ ]:
mview_5, mview_2, mview_10 = dome_vibrational_mode_analysis_helper.get_vibrational_mode_views()

In [ ]:
HBox([mview_2.show(), mview_5.show(), mview_10.show()])

In [ ]:
mview_2.action.play(), mview_5.action.play(), mview_10.action.play()

In [ ]:
mview_5.mode_selector.value = mview_5.mode_selector.options[6]